In [20]:
from collections import Counter

import numpy as np

from utilities import generate_kmers, NUCLEOTIDES

In [4]:
nucl2ind = {nucleotide: i for i, nucleotide in enumerate(NUCLEOTIDES)}

In [43]:
def profile_most_probable_kmer(dna_string, k, profile):
    max_prob = 0
    most_prob_kmer = dna_string[0:k]
    for kmer in generate_kmers(dna_string, k):
        prob = np.prod([profile[nucl2ind[nucl],i] for i, nucl in enumerate(kmer)])
        if prob > max_prob:
            max_prob = prob
            most_prob_kmer = kmer
    return most_prob_kmer

In [11]:
dna_string = "ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT"
k = 5
profile = np.array([
    [0.2, 0.2, 0.3, 0.2, 0.3],
    [0.4, 0.3, 0.1, 0.5, 0.1],
    [0.3, 0.3, 0.5, 0.2, 0.4],
    [0.1, 0.2, 0.1, 0.1, 0.2]
])

In [12]:
assert profile_most_probable_kmer(dna_string, k, profile) == 'CCGAG'

In [13]:
with open("./datasets/dataset_159_3.txt") as fin:
    profile = []
    for i, line in enumerate(fin):
        if i==0:
            dna_string = line.strip()
        elif i==1:
            k = int(line)
        else:
            profile.append([float(f) for f in line.strip().split(" ")])
    profile = np.array(profile)

In [14]:
profile_most_probable_kmer(dna_string, k, profile)

'CACAATACCGTACA'

In [22]:
def init_motif_matrix(dna_strings, k, t):
    motif_matrix = []
    for i in range(t):
        motif_matrix.append(dna_strings[i][:k])
    return motif_matrix


def create_profile(motifs):
    profile = np.zeros((4,len(motifs[0])))
    for coli in range(len(motifs[0])):
        for rowi in range(len(motifs)):
            i = nucl2ind[motifs[rowi][coli]]
            profile[i, coli] += 1/len(motifs)
    return profile

def score_motifs(motifs):
    score = 0
    for coli in range(len(motifs[0])):
        c = Counter()
        for rowi in range(len(motifs)):    
            c[motifs[rowi][coli]] += 1
        most_frequent_nucl = max(c.items(), key=lambda x: x[1])[0]
        c.pop(most_frequent_nucl)
        score += sum(c.values())
    return score

In [23]:
create_profile([
    "AA",
    "AC",
    "TG"
])

array([[0.66666667, 0.33333333],
       [0.        , 0.33333333],
       [0.        , 0.33333333],
       [0.33333333, 0.        ]])

In [24]:
score_motifs([
    "AA",
    "AC",
    "TG"
])

3

In [48]:
def greedy_motif_search(dna_strings, k):
    t = len(dna_strings)
    best_motifs = init_motif_matrix(dna_strings, k, t)
    best_score = len(dna_strings)*len(dna_strings[0])
    for kmer_motif in generate_kmers(dna_strings[0], k):
        motif_matrix = []
        motif_matrix.append(kmer_motif)
        for i in range(1, t):
            profile = create_profile(motif_matrix[0:i])
            most_prob_kmer = profile_most_probable_kmer(dna_strings[i], k, profile)
            motif_matrix.append(most_prob_kmer)
        score = score_motifs(motif_matrix)
        if score < best_score:
            best_score = score
            best_motifs = motif_matrix
    return best_motifs

In [49]:
k=3
dna_strings = [
    "GGCGTTCAGGCA",
    "AAGAATCAGTCA",
    "CAAGGAGTTCGC",
    "CACGTCAATCAC",
    "CAATAATATTCG"
]

In [50]:
greedy_motif_search(dna_strings, k)

['CAG', 'CAG', 'CAA', 'CAA', 'CAA']

In [53]:
with open("./datasets/dataset_159_5.txt") as fin:
    dna_strings = []
    for i, line in enumerate(fin):
        if i==0:
            k, t = line.strip().split(" ")
            k = int(k)
        else:
            dna_strings.append(line.strip())
   

In [54]:
for motif in greedy_motif_search(dna_strings, k):
    print(motif)

TCAAATCGGGTA
CAGAAAAGTGGA
AAGCGGCTGCAC
AACGGACTTCCT
TCCCAGCGGGCA
ATGGAGAAAAAC
TCCGATCGAGGT
TCCCAACGGGAC
TCGGGGCTGCCT
TTCCGAAATGTC
TGTTCGCTCCCA
TGGCAAATGCCA
ACGCAACTAGGT
TCCGAACGTGTC
TCGAAACGTGAA
ACTGAAAAGCTA
AATGAGCTGGAC
ACCCAAAGTGGC
TGGCAGAATGGT
TCGAAGCGTGCA
TGCAATCTGGCT
TCGCAACGCGGA
TCGGATCGGGGC
TCCGAACGAGTT
TCTAAACGGGGT
